In [1]:
#basic
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

#dataset/dataloader stuff
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.io import read_image
import torchvision.transforms as transforms
from torchvision.io import ImageReadMode

#network stuff
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [10]:
!git clone -b david https://github.com/Sioncoolwijk/aml-birds

Cloning into 'aml-birds'...
remote: Enumerating objects: 66915, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 66915 (delta 24), reused 15 (delta 13), pack-reused 66878
Receiving objects: 100% (66915/66915), 1.66 GiB | 31.63 MiB/s, done.
Resolving deltas: 100% (40/40), done.
Updating files: 100% (35422/35422), done.


In [2]:
#pointless now
#datasets dont need gpus to function but usually on top
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
RANDOM_SEED = 42
DEVICE

'cuda'

In [12]:
class LeBirbs(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.image_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_labels.iloc[idx]['image_path']
        img_path = img_path[1:]
        img_path = os.path.join(self.image_dir,img_path)
        image = read_image(img_path, mode = ImageReadMode.RGB)
        image = image.to(torch.float)
        label = self.img_labels.iloc[idx]['label']
        label = torch.tensor(label)

        if self.transform:
            transform = transforms.transforms.CenterCrop(250)
            image = transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [16]:
os.getcwd()

'/content'

In [31]:
#initiate the dataset object
train_dataset = LeBirbs('aml-birds/train_images.csv','aml-birds/train_images',True,None)
test_dataset = LeBirbs('aml-birds/test_images_path.csv','aml-birds/test_images',True,None)
train_dataloader = DataLoader(train_dataset,batch_size=256,shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=256,shuffle=False)

In [36]:
#implementation without atributes
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(3, 3)
        self.conv2 = nn.Conv2d(6, 12, 5)
        self.conv3 = nn.Conv2d(12, 24, 5)
        self.fc1 = nn.Linear(1176,600)
        self.fc2 = nn.Linear(600, 400)
        self.fc3 = nn.Linear(400, 201)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net.to(DEVICE)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=1176, out_features=600, bias=True)
  (fc2): Linear(in_features=600, out_features=400, bias=True)
  (fc3): Linear(in_features=400, out_features=201, bias=True)
)

In [37]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
num_epochs = 50

In [39]:
%%time
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(DEVICE), data[1].to(DEVICE)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print(epoch,running_loss)

print('Finished Training')

0 649.256600856781
1 629.399905204773
2 579.19140625
3 525.0791459083557
4 471.5926480293274
5 416.9112524986267
6 353.41255593299866
7 293.21342527866364
8 227.9541915655136
9 165.88461327552795
10 120.37806403636932
11 83.69715029001236
12 59.985719949007034
13 49.156539529562
14 33.16792377829552
15 27.504681885242462
16 21.58987747132778
17 20.509274311363697
18 17.125190693885088
19 13.692359331995249
20 11.63356557302177
21 9.30772091448307
22 11.144518665969372
23 7.776638574898243
24 4.493235412985086
25 5.920557617675513
26 8.34520659595728
27 6.54775760229677
28 2.8523155555594712
29 1.5777301623020321
30 2.165990208624862
31 1.374693696736358
32 1.9110475837951526
33 0.7575094288913533
34 0.394997204784886
35 0.05878596230468247
36 0.03620086922455812
37 0.022020999262167607
38 0.018741902906185715
39 0.016585022964136442
40 0.014729070600878913
41 0.013667295355844544
42 0.012570939903525868
43 0.011818445316748694
44 0.011061149616580224
45 0.010516602686038823
46 0.009962

In [ ]:
#can save trained models, too large file size for github
#PATH = 'Nets/DK_1_net.pth'
#torch.save(net.state_dict(), PATH)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [40]:
test_file = pd.read_csv('aml-birds/test_images_sample.csv')

all_predictions = []
with torch.no_grad():
    for data in test_dataloader:
        inputs, labels = data[0].to(DEVICE), data[1].to(DEVICE)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predictions = torch.Tensor.tolist(predicted)
        all_predictions.extend(predictions)

In [41]:
test_file['label'] = all_predictions
test_file.to_csv('aml-birds/test_images_predictions_dk2.csv',index=False)